In [1]:
import qiskit
import qiskit.qasm3
import openqasm3

In [2]:
qc = qiskit.QuantumCircuit(2)
qc.h(0)
qc.barrier()
qc.cx(0, 1)
qc.barrier()
qc.measure_all()

In [3]:
with open('test.qasm', 'w') as f:
    qiskit.qasm3.dump(qc, f)

In [4]:
with open ('test.qasm', 'r') as f:
    qasm = f.read()

print(qasm)

OPENQASM 3.0;
include "stdgates.inc";
bit[2] meas;
qubit[2] q;
h q[0];
barrier q[0], q[1];
cx q[0], q[1];
barrier q[0], q[1];
barrier q[0], q[1];
meas[0] = measure q[0];
meas[1] = measure q[1];



In [5]:
qasm_ast_program = openqasm3.ast.Program(qasm)
#openqasm3.parse(qasm)

In [6]:
qasm_ast_program.span

In [7]:
 qasm_parsed_file= openqasm3.parse(qasm)

In [8]:
qasm_parsed_file.span

Span(start_line=1, start_column=0, end_line=11, end_column=22)

In [9]:
qasm_parsed_file.statements

[Include(span=Span(start_line=2, start_column=0, end_line=2, end_column=22), annotations=[], filename='stdgates.inc'),
 ClassicalDeclaration(span=Span(start_line=3, start_column=0, end_line=3, end_column=11), annotations=[], type=BitType(span=Span(start_line=3, start_column=0, end_line=3, end_column=5), size=IntegerLiteral(span=Span(start_line=3, start_column=3, end_line=3, end_column=5), value=2)), identifier=Identifier(span=Span(start_line=3, start_column=45, end_line=3, end_column=48), name='meas'), init_expression=None),
 QubitDeclaration(span=Span(start_line=4, start_column=0, end_line=4, end_column=10), annotations=[], qubit=Identifier(span=Span(start_line=4, start_column=60, end_line=4, end_column=60), name='q'), size=IntegerLiteral(span=Span(start_line=4, start_column=5, end_line=4, end_column=7), value=2)),
 QuantumGate(span=Span(start_line=5, start_column=0, end_line=5, end_column=6), annotations=[], modifiers=[], name=Identifier(span=Span(start_line=5, start_column=63, end_l

In [10]:
qasm_parsed_file.version

'3.0'

In [11]:
state = openqasm3.visitor.QASMVisitor().visit(node=qasm_parsed_file)

In [12]:
state

In [13]:
qasm_parsed_file.statements

[Include(span=Span(start_line=2, start_column=0, end_line=2, end_column=22), annotations=[], filename='stdgates.inc'),
 ClassicalDeclaration(span=Span(start_line=3, start_column=0, end_line=3, end_column=11), annotations=[], type=BitType(span=Span(start_line=3, start_column=0, end_line=3, end_column=5), size=IntegerLiteral(span=Span(start_line=3, start_column=3, end_line=3, end_column=5), value=2)), identifier=Identifier(span=Span(start_line=3, start_column=45, end_line=3, end_column=48), name='meas'), init_expression=None),
 QubitDeclaration(span=Span(start_line=4, start_column=0, end_line=4, end_column=10), annotations=[], qubit=Identifier(span=Span(start_line=4, start_column=60, end_line=4, end_column=60), name='q'), size=IntegerLiteral(span=Span(start_line=4, start_column=5, end_line=4, end_column=7), value=2)),
 QuantumGate(span=Span(start_line=5, start_column=0, end_line=5, end_column=6), annotations=[], modifiers=[], name=Identifier(span=Span(start_line=5, start_column=63, end_l

In [14]:
for statement in qasm_parsed_file.statements:
    print(statement)

Include(span=Span(start_line=2, start_column=0, end_line=2, end_column=22), annotations=[], filename='stdgates.inc')
ClassicalDeclaration(span=Span(start_line=3, start_column=0, end_line=3, end_column=11), annotations=[], type=BitType(span=Span(start_line=3, start_column=0, end_line=3, end_column=5), size=IntegerLiteral(span=Span(start_line=3, start_column=3, end_line=3, end_column=5), value=2)), identifier=Identifier(span=Span(start_line=3, start_column=45, end_line=3, end_column=48), name='meas'), init_expression=None)
QubitDeclaration(span=Span(start_line=4, start_column=0, end_line=4, end_column=10), annotations=[], qubit=Identifier(span=Span(start_line=4, start_column=60, end_line=4, end_column=60), name='q'), size=IntegerLiteral(span=Span(start_line=4, start_column=5, end_line=4, end_column=7), value=2))
QuantumGate(span=Span(start_line=5, start_column=0, end_line=5, end_column=6), annotations=[], modifiers=[], name=Identifier(span=Span(start_line=5, start_column=63, end_line=5, 

In [15]:
for statement in qasm_parsed_file.statements:
    if isinstance(statement, openqasm3.ast.Include):
        print('Included filenames:', statement.filename)
    elif isinstance(statement, openqasm3.ast.ClassicalDeclaration):
        if isinstance(statement.type, openqasm3.ast.BitType):
            cbit_type = 'bit'
        else:
            raise ValueError('Unknown type', 'statement.type:', statement.type)
        cbit_name = statement.identifier.name
        cbit_length = statement.type.size.value
        print('Classical bit name:', cbit_name)
        print('Classical bit type:', cbit_type)
        print('Classical bit length:', cbit_length)
    elif isinstance(statement, openqasm3.ast.QubitDeclaration):
        qbit_name = statement.qubit.name
        qbit_length = statement.size.value
        print('Quantum bit name:', qbit_name)
        print('Quantum bit length:', qbit_length)
    elif isinstance(statement, openqasm3.ast.QuantumGate):
        qgate_name = statement.name.name
        qgate_qbit_names = [statement.qubits[i].name.name for i in range(len(statement.qubits))]
        qgate_qbit_indices = [statement.qubits[i].indices[0][0].value for i in range(len(statement.qubits))] # why is there doubly nested list in indices?
        print('Quantum gate name:', qgate_name)
        print('Quantum gate qubits names:', qgate_qbit_names)
        print('Quantum gate qubits indices:', qgate_qbit_indices)
    elif isinstance(statement, openqasm3.ast.QuantumBarrier):
        #print(statement)
        qbarrier_name = 'barrier' # name is not stored in statement 
        qbarrier_qbit_names = [statement.qubits[i].name.name for i in range(len(statement.qubits))]
        qbarrier_qbit_indices = [statement.qubits[i].indices[0][0].value for i in range(len(statement.qubits))] # why is there doubly nested list in indices?
        print('Quantum gate name:', qbarrier_name)
        print('Quantum gate qubits names:', qbarrier_qbit_names)
        print('Quantum gate qubits indices:', qbarrier_qbit_indices)
    elif isinstance(statement, openqasm3.ast.QuantumMeasurementStatement):
        #print(statement)
        qmeasurement_name = 'measurement' # name is not stored in statement 
        qmeasurement_qbit_name = statement.measure.qubit.name.name
        qmeasurement_qbit_index = statement.measure.qubit.indices[0][0].value
        qmeasurement_cbit_name = statement.target.name.name
        qmeasurement_cbit_index = statement.target.indices[0][0].value
        print('Quantum measurement name:', qmeasurement_name)
        print('Quantum measurement qubit name:', qmeasurement_qbit_name)
        print('Quantum measurement qubit index:', qmeasurement_qbit_index)
        print('Quantum measurement cbit name:', qmeasurement_cbit_name)
        print('Quantum measurement cbit index:', qmeasurement_cbit_index)

######
#
# the statements are ordered in some logical timeseries manner. A gate can be interpreted as a definite border between two time steps.
#
######

Included filenames: stdgates.inc
Classical bit name: meas
Classical bit type: bit
Classical bit length: 2
Quantum bit name: q
Quantum bit length: 2
Quantum gate name: h
Quantum gate qubits names: ['q']
Quantum gate qubits indices: [0]
Quantum gate name: barrier
Quantum gate qubits names: ['q', 'q']
Quantum gate qubits indices: [0, 1]
Quantum gate name: cx
Quantum gate qubits names: ['q', 'q']
Quantum gate qubits indices: [0, 1]
Quantum gate name: barrier
Quantum gate qubits names: ['q', 'q']
Quantum gate qubits indices: [0, 1]
Quantum gate name: barrier
Quantum gate qubits names: ['q', 'q']
Quantum gate qubits indices: [0, 1]
Quantum measurement name: measurement
Quantum measurement qubit name: q
Quantum measurement qubit index: 0
Quantum measurement cbit name: meas
Quantum measurement cbit index: 0
Quantum measurement name: measurement
Quantum measurement qubit name: q
Quantum measurement qubit index: 1
Quantum measurement cbit name: meas
Quantum measurement cbit index: 1


In [16]:
import QSymPy

In [17]:
qcs = QSymPy.QuantumCircuit(num_qubits=2, num_clbits=2)

In [ ]:
qcs.add_gate('H', qubits_t=[0], step=0)
qcs.add_barrier(step=0)
qcs.add_gate('CNOT', qubits_c=[0], qubits_t=[1], step=1)
qcs.add_barrier(step=1)

qcs.assemble_unitary()
qcs.unitary


Matrix([
[CNOT_I_qt1_qc0_s2_p00*H_qt0_qcs1_p00, CNOT_I_qt1_qc0_s2_p00*H_qt0_qcs1_p01, CNOT_I_qt1_qc0_s2_p01*H_qt0_qcs1_p00, CNOT_I_qt1_qc0_s2_p01*H_qt0_qcs1_p01],
[CNOT_X_qt1_qc0_s2_p00*H_qt0_qcs1_p10, CNOT_X_qt1_qc0_s2_p00*H_qt0_qcs1_p11, CNOT_X_qt1_qc0_s2_p01*H_qt0_qcs1_p10, CNOT_X_qt1_qc0_s2_p01*H_qt0_qcs1_p11],
[CNOT_I_qt1_qc0_s2_p10*H_qt0_qcs1_p00, CNOT_I_qt1_qc0_s2_p10*H_qt0_qcs1_p01, CNOT_I_qt1_qc0_s2_p11*H_qt0_qcs1_p00, CNOT_I_qt1_qc0_s2_p11*H_qt0_qcs1_p01],
[CNOT_X_qt1_qc0_s2_p10*H_qt0_qcs1_p10, CNOT_X_qt1_qc0_s2_p10*H_qt0_qcs1_p11, CNOT_X_qt1_qc0_s2_p11*H_qt0_qcs1_p10, CNOT_X_qt1_qc0_s2_p11*H_qt0_qcs1_p11]])

In [19]:
qcs.barrier_collection

[Barrier at step 1, Barrier at step 2]

In [20]:
qc_imported = QSymPy.openqasm3_to_qc('test.qasm')

Included filenames: stdgates.inc
Classical bit name: meas
Classical bit type: bit
Classical bit length: 2
Quantum bit name: q
Quantum bit length: 2
Quantum gate name: h
Quantum gate qubits names: ['q']
Quantum gate qubits indices: [0]
Quantum gate name: barrier
Quantum gate qubits names: ['q', 'q']
Quantum gate qubits indices: [0, 1]
Quantum gate name: cx
Quantum gate qubits names: ['q', 'q']
Quantum gate qubits indices: [0, 1]
Quantum gate name: barrier
Quantum gate qubits names: ['q', 'q']
Quantum gate qubits indices: [0, 1]
Quantum gate name: barrier
Quantum gate qubits names: ['q', 'q']
Quantum gate qubits indices: [0, 1]
Quantum measurement name: measurement
Quantum measurement qubit name: q
Quantum measurement qubit index: 0
Quantum measurement cbit name: meas
Quantum measurement cbit index: 0
Quantum measurement name: measurement
Quantum measurement qubit name: q
Quantum measurement qubit index: 1
Quantum measurement cbit name: meas
Quantum measurement cbit index: 1


In [21]:
qc_imported.assemble_unitary()
qc_imported.unitary

Matrix([
[CNOT_I_qt1_qc0_s1_p00*H_qt0_qcs0_p00, CNOT_I_qt1_qc0_s1_p00*H_qt0_qcs0_p01, CNOT_I_qt1_qc0_s1_p01*H_qt0_qcs0_p00, CNOT_I_qt1_qc0_s1_p01*H_qt0_qcs0_p01],
[CNOT_X_qt1_qc0_s1_p00*H_qt0_qcs0_p10, CNOT_X_qt1_qc0_s1_p00*H_qt0_qcs0_p11, CNOT_X_qt1_qc0_s1_p01*H_qt0_qcs0_p10, CNOT_X_qt1_qc0_s1_p01*H_qt0_qcs0_p11],
[CNOT_I_qt1_qc0_s1_p10*H_qt0_qcs0_p00, CNOT_I_qt1_qc0_s1_p10*H_qt0_qcs0_p01, CNOT_I_qt1_qc0_s1_p11*H_qt0_qcs0_p00, CNOT_I_qt1_qc0_s1_p11*H_qt0_qcs0_p01],
[CNOT_X_qt1_qc0_s1_p10*H_qt0_qcs0_p10, CNOT_X_qt1_qc0_s1_p10*H_qt0_qcs0_p11, CNOT_X_qt1_qc0_s1_p11*H_qt0_qcs0_p10, CNOT_X_qt1_qc0_s1_p11*H_qt0_qcs0_p11]])